# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc826a18370>
├── 'a' --> tensor([[ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724]])
└── 'x' --> <FastTreeValue 0x7fc826a184f0>
    └── 'c' --> tensor([[ 0.4296, -0.3336, -0.1647,  0.3837],
                        [ 1.3859, -1.0740,  1.8592,  0.5435],
                        [-1.2189, -1.0393, -0.1546, -0.8087]])

In [4]:
t.a

tensor([[ 0.1401,  0.1862, -0.2312],
        [-0.2158,  0.0365,  0.5724]])

In [5]:
%timeit t.a

73.1 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc826a18370>
├── 'a' --> tensor([[ 0.6394, -0.1007, -0.4428],
│                   [ 1.4431, -0.3313,  2.4880]])
└── 'x' --> <FastTreeValue 0x7fc826a184f0>
    └── 'c' --> tensor([[ 0.4296, -0.3336, -0.1647,  0.3837],
                        [ 1.3859, -1.0740,  1.8592,  0.5435],
                        [-1.2189, -1.0393, -0.1546, -0.8087]])

In [7]:
%timeit t.a = new_value

81.4 ns ± 0.106 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1401,  0.1862, -0.2312],
               [-0.2158,  0.0365,  0.5724]]),
    x: Batch(
           c: tensor([[ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1401,  0.1862, -0.2312],
        [-0.2158,  0.0365,  0.5724]])

In [11]:
%timeit b.a

68.1 ns ± 0.0531 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5343, -0.1484, -0.2327],
               [ 1.2595,  0.1501,  0.1004]]),
    x: Batch(
           c: tensor([[ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087]]),
       ),
)

In [13]:
%timeit b.a = new_value

560 ns ± 0.121 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

941 ns ± 7.75 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 27.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 904 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc756bf6cd0>
├── 'a' --> tensor([[[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]],
│           
│                   [[ 0.1401,  0.1862, -0.2312],
│                    [-0.2158,  0.0365,  0.5724]]])
└── 'x' --> <FastTreeValue 0x7fc756bf5f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 65.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc75ed0deb0>
├── 'a' --> tensor([[ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724],
│                   [ 0.1401,  0.1862, -0.2312],
│                   [-0.2158,  0.0365,  0.5724]])
└── 'x' --> <FastTreeValue 0x7fc75ed0df40>
    └── 'c' --> tensor([[ 0.4296, -0.3336, -0.1647,  0.3837],
                        [ 1.3859, -1.0740,  1.8592,  0.5435],
                 

In [23]:
%timeit t_cat(trees)

31.2 µs ± 59.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.1 µs ± 28.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.4296, -0.3336, -0.1647,  0.3837],
                       [ 1.3859, -1.0740,  1.8592,  0.5435],
                       [-1.2189, -1.0393, -0.1546, -0.8087]],
              
                      [[ 0.4296, -0.3336, -0.1647,  0.3837],
                       [ 1.3859, -1.0740,  1.8592,  0.5435],
                       [-1.2189, -1.0393, -0.1546, -0.8087]],
              
                      [[ 0.4296, -0.3336, -0.1647,  0.3837],
                       [ 1.3859, -1.0740,  1.8592,  0.5435],
                       [-1.2189, -1.0393, -0.1546, -0.8087]],
              
                      [[ 0.4296, -0.3336, -0.1647,  0.3837],
                       [ 1.3859, -1.0740,  1.8592,  0.5435],
                       [-1.2189, -1.0393, -0.1546, -0.8087]],
              
                      [[ 0.4296, -0.3336, -0.1647,  0.3837],
                       [ 1.3859, -1.0740,  1.8592,  0.5435],
                       [-1.2189, -1.0393, -0.1546, -0.8087]],

In [26]:
%timeit Batch.stack(batches)

83.9 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087],
                      [ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087],
                      [ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087],
                      [ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087],
                      [ 0.4296, -0.3336, -0.1647,  0.3837],
                      [ 1.3859, -1.0740,  1.8592,  0.5435],
                      [-1.2189, -1.0393, -0.1546, -0.8087],
                      [ 0.4296, -0.3336, -0.1647,  0.3837],
                   

In [28]:
%timeit Batch.cat(batches)

155 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

349 µs ± 7.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
